# Module 2 - Moving Bits Around (with OpenCL)

## Setup

### Library Import
Before doing anything else, we need to import [PyOpenCL](https://documen.tician.de/pyopencl/) and [NumPy](http://www.numpy.org/).

In [ ]:
import pyopencl,numpy

### Setting up platforms, devices and context
We're going to setup the devices and context as explicit objects because we might want to interogate their runtime information.

In [ ]:
platforms = pyopencl.get_platforms()
nvidia_device,intel_device = [platform.get_devices()[0] 
                              for platform in platforms]
nvidia_context,intel_context = [pyopencl.Context(devices=[device]) 
                                for device in (nvidia_device,intel_device)]

## Communicating between Host and Device
### Setting up the program
1. Create a program for Vector element-wise multiplication
2. Compile the programs

In [ ]:
program_source = """
kernel void square(global long *b)
{
  int gid = get_global_id(0);
  
  b[gid] = b[gid]*b[gid];
}

kernel void operation(global long *a,
                      global long *b)
{
  int gid = get_global_id(0);
  
  long a_temp = a[gid];
  long b_temp = b[gid];
  
  b[gid] = b_temp/a_temp + b_temp*a_temp - b_temp%a_temp;
}
"""
nvidia_program_source,intel_program_source = [pyopencl.Program(context,program_source) 
                                              for context in (nvidia_context,intel_context)]

In [ ]:
nvidia_program,intel_program = [program.build()
                                for program 
                                in (nvidia_program_source,
                                    intel_program_source)]

### Creating the global memory resource
1. Defining source data parameters
2. Creating the source data
3. Creating the memory resources within the context

In [ ]:
M = 100
N = int(64e2)
dt = numpy.int64
dt_size = numpy.dtype(dt).itemsize

In [ ]:
a = numpy.random.randint(low=1,high=10,size=(M,N)).astype(dt)
b = numpy.random.randint(low=1,high=1000,size=(M,N)).astype(dt)*a

In [ ]:
def create_buffers(context,a_size,b_size):
    mem_flags = pyopencl.mem_flags.READ_ONLY | pyopencl.mem_flags.ALLOC_HOST_PTR
    a_buffer = pyopencl.Buffer(context,
                               flags = mem_flags, 
                               size = a_size)
    b_buffer = pyopencl.Buffer(context, 
                               flags = mem_flags,                               
                               size = b_size)
    return a_buffer,b_buffer

In [ ]:
nvidia_a_buffer,nvidia_b_buffer = create_buffers(nvidia_context,
                                                 N*dt_size,
                                                 N*dt_size)
intel_a_buffer,intel_b_buffer = create_buffers(intel_context,
                                               N*dt_size,
                                               N*dt_size)

## Running the program
### Defining the host program

In [ ]:
def compute_norm(queue,a,a_buffer,b,b_buffer,program):
    c = numpy.empty_like(a)
    total = 0.0
    i = 0
    for i in range(M):
        a_row = a[i]
        b_row = b[i]
        #for i,(a_row,b_row) in enumerate(zip(a,b)):
        #copying data onto device
        copyon_events = [pyopencl.enqueue_copy(queue,
                                                src=a_row,
                                                dest=a_buffer,
                                                is_blocking = False),
                         pyopencl.enqueue_copy(queue,
                                                src=b_row,
                                                dest=b_buffer,
                                                is_blocking = False)]
        
        #running program
        kernel_event = program.operation(queue,
                                         a_row.shape, #global size
                                         None, #local size
                                         a_buffer,b_buffer,
                                         wait_for = copyon_events)
        
        kernel_event2 = program.square(queue,
                                       b_row.shape, #global size
                                       None, #local size
                                       b_buffer,
                                       wait_for = [kernel_event])
        
        #copying data off device
        copyoff_event = pyopencl.enqueue_copy(queue,
                                              src = b_buffer,
                                              dest = c[i],
                                              wait_for = [kernel_event2],
                                              is_blocking = False)
            
        #wait for copy-off to finish
        copyoff_event.wait()
        i += 1
        
    total = c.sum()
        
    return total**0.5

### In-order Execution
1. In-order queue
2. Computing the norm
3. Checking the result

In [ ]:
nvidia_io_queue = pyopencl.CommandQueue(nvidia_context)
intel_io_queue = pyopencl.CommandQueue(intel_context)

In [ ]:
nvidia_io_norm = compute_norm(nvidia_io_queue,
                              a,nvidia_a_buffer,
                              b,nvidia_b_buffer,
                              nvidia_program)

intel_io_norm = compute_norm(intel_io_queue,
                              a,intel_a_buffer,
                              b,intel_b_buffer,
                              intel_program)

In [ ]:
reference_result = numpy.linalg.norm(b/a + b*a - b%a)

In [ ]:
if(reference_result - nvidia_io_norm > 0): raise Exception("nvidia result does not match!")
if(reference_result - intel_io_norm > 0): raise Exception("intel result does not match!")

## Out-of-order Execution
Similiar to before, but using out of order execution

In [ ]:
nvidia_oo_queue = pyopencl.CommandQueue(nvidia_context,
                                        properties = pyopencl.command_queue_properties.OUT_OF_ORDER_EXEC_MODE_ENABLE)
intel_oo_queue = pyopencl.CommandQueue(intel_context,
                                       properties = pyopencl.command_queue_properties.OUT_OF_ORDER_EXEC_MODE_ENABLE)

In [ ]:
nvidia_oo_norm = compute_norm(nvidia_oo_queue,
                              a,nvidia_a_buffer,
                              b,nvidia_b_buffer,
                              nvidia_program)
    
intel_oo_norm = compute_norm(intel_oo_queue,
                             a,intel_a_buffer,
                             b,intel_b_buffer,
                             intel_program)

In [ ]:
if(reference_result - nvidia_oo_norm > 0): raise Exception("nvidia result does not match!")
if(reference_result - intel_oo_norm > 0): raise Exception("intel result does not match!")

### Performance Comparison

In [ ]:
%timeit -n 1 compute_norm(nvidia_io_queue,a,nvidia_a_buffer,b,nvidia_b_buffer,nvidia_program)
%timeit -n 1 compute_norm(intel_io_queue,a,intel_a_buffer,b,intel_b_buffer,intel_program)

In [ ]:
%timeit -n 1 compute_norm(nvidia_oo_queue,a,nvidia_a_buffer,b,nvidia_b_buffer,nvidia_program)
%timeit -n 1 compute_norm(intel_oo_queue,a,intel_a_buffer,b,intel_b_buffer,intel_program)

In [ ]:
%timeit -n 1 numpy.linalg.norm(b/a + b*a - b%a)

## Helping the device talk to itself

## Defining the program
1. Compiling the new program
2. Rewriting the calling code

In [ ]:
program_source_local = """
#define WG_SIZE 128
kernel void operation_local(global long *a,
                            global long *b,
                             local long *c)
{
  int gid = get_global_id(0);
  int lid = get_local_id(0);
  
  long a_temp = a[gid];
  long b_temp = b[gid];
  
  long result = b_temp/a_temp + b_temp*a_temp - b_temp%a_temp;
  c[lid] = result * result;
  
  barrier(CLK_LOCAL_MEM_FENCE);
  
  long sum = 0;
  if(lid==0){
      int wgid = get_group_id(0);
      for(int i=0;i<WG_SIZE;++i) sum += c[i];
      b[wgid] = sum;
  }
}
"""
nvidia_program_source,intel_program_source = [pyopencl.Program(context,program_source_local) 
                                              for context in (nvidia_context,intel_context)]
nvidia_program_local,intel_program_local = [program.build()
                                            for program 
                                            in (nvidia_program_source,intel_program_source)]

In [ ]:
def compute_norm_local(queue,a,a_buffer,b,b_buffer,program):
    WG_SIZE = 128
    wgs = int(len(a[0])/WG_SIZE)
    
    c = numpy.empty((len(a),wgs),dtype=dt)
    total = 0.0
    for i,(a_row,b_row) in enumerate(zip(a,b)):
        #copying data onto device
        copyon_events = []
        
        copyon_events += [pyopencl.enqueue_copy(queue,
                                                src=a_row,
                                                dest=a_buffer,
                                                is_blocking = False)]
        copyon_events += [pyopencl.enqueue_copy(queue,
                                                src=b_row,
                                                dest=b_buffer,
                                                is_blocking = False)]
        
        #running program
        local_c = pyopencl.LocalMemory(WG_SIZE*dt_size)
        kernel_event = program.operation_local(queue,
                                               a_row.shape, #global size
                                               (WG_SIZE,), #local size
                                               a_buffer,b_buffer,local_c,
                                               wait_for = copyon_events)
        
        
        #copying data off device
        copyoff_event = pyopencl.enqueue_copy(queue,
                                              src = b_buffer,
                                              dest = c[i],
                                              wait_for = [kernel_event],
                                              is_blocking = False)
        
        #since we might as well do something useful while we wait
        if(i>0): total += c[i-1].sum()
            
        #wait for copy-off to finish
        copyoff_event.wait()
        
    total += c[-1].sum()
        
    return total**0.5

## Checking the result
1. Checking with out of order execution queues
2. Evaluating performance

In [ ]:
nvidia_oo_norm_local = compute_norm_local(nvidia_oo_queue,
                                          a,nvidia_a_buffer,
                                          b,nvidia_b_buffer,
                                          nvidia_program_local)

if(reference_result - nvidia_oo_norm_local > 0):
    raise Exception("nvidia result does not match!")
    
intel_oo_norm_local = compute_norm_local(intel_oo_queue,
                                         a,intel_a_buffer,
                                         b,intel_b_buffer,
                                         intel_program_local)

if(reference_result - intel_oo_norm_local > 0): raise Exception("intel result does not match!")

In [ ]:
%timeit -n 10 compute_norm_local(nvidia_oo_queue,a,nvidia_a_buffer,b,nvidia_b_buffer,nvidia_program_local)
%timeit -n 10 compute_norm_local(intel_oo_queue,a,intel_a_buffer,b,intel_b_buffer,intel_program_local)

## Module Challenge
* Perform matrix multiplication using global, local and constant memory. 
* Measure the performance difference between the three.

*Hint: Take advantage of multiple indices.*

In [ ]:
#Setting up memory
M = 1024
N = 8192
dt = numpy.float32
dt_size = numpy.dtype(dt).itemsize
a = numpy.random.random(size=(M,N)).astype(dt)
b = numpy.asfortranarray(numpy.random.random(size=(N,M)).astype(dt))

In [ ]:
program_source = """
#define ROWS %d
#define COLS %d

kernel void opencl_dot(global float *a,
                       global float *b,
                       global float *c)
{
    int row = get_global_id(0);
    int col = get_global_id(1);

    int a_offset = row*COLS;
    int b_offset = col*COLS;

    float sum = 0;
    for(int i=0;i<COLS;++i) sum += a[a_offset+i] * b[b_offset+i];

    int c_index = row*ROWS + col;
    c[c_index] = sum;
}
"""%(M,N)
nvidia_program_source,intel_program_source = [pyopencl.Program(context,program_source) 
                                              for context 
                                              in (nvidia_context,intel_context)]
nvidia_program,intel_program = [program.build()
                                for program
                                in (nvidia_program_source,intel_program_source)]

In [ ]:
#Creating buffers
nvidia_a_buffer,nvidia_b_buffer = create_buffers(nvidia_context,M*N*dt_size,M*N*dt_size)
intel_a_buffer,intel_b_buffer = create_buffers(intel_context,M*N*dt_size,M*N*dt_size)

nvidia_c_buffer = pyopencl.Buffer(nvidia_context,
                           flags=pyopencl.mem_flags.WRITE_ONLY, 
                           size=M*M*dt_size)
intel_c_buffer = pyopencl.Buffer(intel_context,
                           flags=pyopencl.mem_flags.WRITE_ONLY, 
                           size=M*M*dt_size)

In [ ]:
def opencl_dot(queue,a,a_buffer,b,b_buffer,program,c_buffer):
    
    #copying data onto device
    copyon_events = []
        
    copyon_events += [pyopencl.enqueue_copy(queue,
                                            src=a,
                                            dest=a_buffer,
                                            is_blocking = False)]
    copyon_events += [pyopencl.enqueue_copy(queue,
                                            src=b.T,
                                            dest=b_buffer,
                                            is_blocking = False)]
        
    #running program
    kernel_event = program.opencl_dot(queue,
                                      (M,M), #global size
                                      None, #local size
                                      a_buffer,b_buffer,c_buffer,
                                      wait_for = copyon_events)
        
        
    #copying data off device
    c = numpy.empty((M,M),dtype=dt)
    copyoff_event = pyopencl.enqueue_copy(queue,
                                          src = c_buffer,
                                          dest = c,
                                          wait_for = [kernel_event]).wait()
        
    return c

In [ ]:
nvidia_io_queue = pyopencl.CommandQueue(nvidia_context)
nvidia_result = opencl_dot(nvidia_io_queue,a,nvidia_a_buffer,b,nvidia_b_buffer,nvidia_program,nvidia_c_buffer)
%timeit opencl_dot(nvidia_io_queue,a,nvidia_a_buffer,b,nvidia_b_buffer,nvidia_program,nvidia_c_buffer)
nvidia_io_queue.finish()

In [ ]:
intel_io_queue = pyopencl.CommandQueue(intel_context)
intel_result = opencl_dot(intel_io_queue,a,intel_a_buffer,b,intel_b_buffer,intel_program,intel_c_buffer)
%timeit opencl_dot(intel_io_queue,a,intel_a_buffer,b,intel_b_buffer,intel_program,intel_c_buffer)
intel_io_queue.finish()

In [ ]:
%timeit a.dot(b)